In [ ]:
# install packages necessary for project
!pip install covidcast
!pip install pyspark

from datetime import date, timedelta
import covidcast
import pandas as pd

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datetime import date, timedelta
import covidcast
import pandas as pd

# Function to read data from COVIDcast and save to csv
def get_data(state_dict,start_date,end_date,geo_level,data_source_dict):
    data_list = [] # initialize data list
    # for each source of data
    for data_source in data_source_dict:
        # for each signal in the source of data
        for signal in data_source_dict[data_source]:
            data = None # initialize the data
            print("... currently fetching (" + data_source + ", " + signal + ") ...") # print to see progress
            data = covidcast.signal(data_source, signal, start_date, end_date, geo_level)
            data = data.drop(columns=['issue','lag','geo_type']) # drop unnecesary columns
            if 'direction' in data.columns:
              data.drop(columns=['direction'])
            data.columns = ['state_code','signal','date','value','stderr','sample_size',
                            'data_source'] # set column names
            data['date'] = data['date'].astype(str) # convert to string
            data_list.append(data) # add to list of data
    return data_list

# dictionary of all neighboring states
state_neighbors = {'al': ['ms','tn','ga','fl'], 'ar': ['mo','tn','ms','la','tx','ok'],
                   'az': ['ca','nv','ut','nm'], 'ca': ['or','az','nv'], 'co': ['ut','wy','ne','ks','ok','nm'],
                   'ct': ['ri','ma','ny'], 'dc':['va','md'], 'de': ['nj','pa','md'], 'fl': ['ga','al'],
                   'ga': ['fl','al','tn','nc','sc'], 'ia': ['mn','wi','il','mo','ne','sd'],
                   'id': ['wa','or','nv','ut','wy','mt'], 'il': ['wi','ia','mo','ky','in'],
                   'in': ['il','mi','oh','ky'], 'ks': ['co','ok','mo','ne'],
                   'ky': ['il','in','oh','wv','va','tn','mo'], 'la': ['tx','ar','ms'],
                   'ma': ['vt','nh','ct','ri','ny'], 'md': ['pa','de','va','wv','dc'], 'me': ['nh'],
                   'mi': ['wi','in','oh'], 'mn': ['nd','sd','ia','wi'],
                   'mo': ['ks','ne','ia','il','ky','tn','ar','ok'], 'ms': ['ar','la','al','tn'],
                   'mt': ['id','wy','sd','nd'], 'nc': ['va','tn','ga','sc'], 'nd': ['mt','sd','mn'],
                   'ne': ['wy','sd','ia','mo','ks','co'], 'nh': ['vt','me','ma'], 'nj': ['ny','pa','de'],
                   'nm': ['az','co','ok','tx'], 'nv': ['or','id','ut','az','ca'], 'ny':['pa','nj','ct','ma','vt'],
                   'oh': ['mi','in','ky','wv','pa'], 'ok': ['co','ks','mo','ar','tx','nm'],
                   'or': ['wa','id','nv','ca'], 'pa': ['ny','nj','de','md','wv','oh'],
                   'ri': ['ma','ct','ny'], 'sc': ['nc','ga'], 'sd': ['mt','nd','mn','ia','ne','wy'],
                   'tn': ['mo','ky','va','nc','ga','al','ms','ar'], 'tx': ['nm','ok','ar','la'],
                   'ut': ['nv','az','co','wy','id'], 'va': ['wv','ky','tn','nc','dc','md'], 'vt': ['ny','nh','ma'],
                   'wa': ['id','or'], 'wi': ['mn','mi','il','ia'], 'wv': ['oh','ky','pa','md','va'],
                   'wy': ['id','ut','co','ne','sd','mt']}

# dictionary of all state codes and their names
state_dict = {'al': 'Alabama', 'ar': 'Arkansas', 'az': 'Arizona', 'ca': 'California',
              'co': 'Colorado', 'ct': 'Connecticut', 'dc': 'District of Columbia', 'de': 'Delaware',
              'fl': 'Florida', 'ga': 'Georgia', 'ia': 'Iowa', 'id': 'Idaho',
              'il': 'Illinois', 'in': 'Indiana', 'ks': 'Kansas', 'ky': 'Kentucky', 'la': 'Louisiana',
              'ma': 'Massachusetts', 'md': 'Maryland', 'me': 'Maine', 'mi': 'Michigan', 'mn': 'Minnesota',
              'mo': 'Missouri', 'ms': 'Mississippi', 'mt': 'Montana', 'nc': 'North Carolina', 'nd': 'North Dakota',
              'ne': 'Nebraska', 'nh': 'New Hampshire', 'nj': 'New Jersey', 'nm': 'New Mexico', 'nv': 'Nevada',
              'ny': 'New York', 'oh': 'Ohio', 'ok': 'Oklahoma', 'or': 'Oregon', 'pa': 'Pennsylvania',
              'ri': 'Rhode Island', 'sc': 'South Carolina', 'sd': 'South Dakota', 'tn': 'Tennessee', 'tx': 'Texas',
              'ut': 'Utah', 'va': 'Virginia', 'vt': 'Vermont', 'wa': 'Washington', 'wi': 'Wisconsin',
              'wv': 'West Virginia', 'wy': 'Wyoming'}

today_dict = {"indicator-combination": ["confirmed_7dav_incidence_num","deaths_7dav_incidence_num"]}

two_week_dict = {"doctor-visits": ["smoothed_adj_cli"],
                 "ght": ["smoothed_search"],
                 "hospital-admissions": ["smoothed_adj_covid19_from_claims"],
                 "quidel": ["covid_ag_smoothed_pct_positive"],
                 "safegraph": ["full_time_work_prop","part_time_work_prop"],
                 "indicator-combination": ["confirmed_7dav_incidence_num","deaths_7dav_incidence_num"]}

three_week_dict = {"indicator-combination": ["confirmed_7dav_incidence_num","deaths_7dav_incidence_num"]}

four_week_dict = {"indicator-combination": ["confirmed_7dav_incidence_num","deaths_7dav_incidence_num"]}

# set data for calling COVIDcast
start_date = date(2020, 4, 1)
end_date = date.today() - timedelta(days=7)
geo_level = "state"

In [ ]:
today_data_list = get_data(state_dict,date(2020, 4, 1),date.today() - timedelta(days=7),"state",today_dict)
two_week_data_list = get_data(state_dict,date(2020, 3, 18),date.today() - timedelta(days=21),"state",two_week_dict)
three_week_data_list = get_data(state_dict,date(2020, 3, 11),date.today() - timedelta(days=28),"state",three_week_dict)
four_week_data_list = get_data(state_dict,date(2020, 3, 4),date.today() - timedelta(days=35),"state",four_week_dict)
pop_df = pd.read_csv('/content/drive/My Drive/MSDA/MKTG881/data/state_pop_density.csv')
mask_df = pd.read_csv('/content/drive/My Drive/MSDA/MKTG881/data/mask_mandates.csv')
latlon_df = pd.read_csv('/content/drive/My Drive/MSDA/MKTG881/data/latlon.csv')

... currently fetching (indicator-combination, confirmed_7dav_incidence_num) ...
... currently fetching (indicator-combination, deaths_7dav_incidence_num) ...
... currently fetching (doctor-visits, smoothed_adj_cli) ...
... currently fetching (ght, smoothed_search) ...


/usr/local/lib/python3.6/dist-packages/covidcast/covidcast.py:391: NoDataWarning: No ght smoothed_search data found on 20200817 for geography 'state'
  NoDataWarning)
/usr/local/lib/python3.6/dist-packages/covidcast/covidcast.py:391: NoDataWarning: No ght smoothed_search data found on 20200821 for geography 'state'
  NoDataWarning)
/usr/local/lib/python3.6/dist-packages/covidcast/covidcast.py:391: NoDataWarning: No ght smoothed_search data found on 20200919 for geography 'state'
  NoDataWarning)
/usr/local/lib/python3.6/dist-packages/covidcast/covidcast.py:391: NoDataWarning: No ght smoothed_search data found on 20201103 for geography 'state'
  NoDataWarning)
/usr/local/lib/python3.6/dist-packages/covidcast/covidcast.py:391: NoDataWarning: No ght smoothed_search data found on 20201104 for geography 'state'
  NoDataWarning)
/usr/local/lib/python3.6/dist-packages/covidcast/covidcast.py:391: NoDataWarning: No ght smoothed_search data found on 20201105 for geography 'state'
  NoDataWarning

... currently fetching (hospital-admissions, smoothed_adj_covid19_from_claims) ...
... currently fetching (quidel, covid_ag_smoothed_pct_positive) ...


/usr/local/lib/python3.6/dist-packages/covidcast/covidcast.py:391: NoDataWarning: No quidel covid_ag_smoothed_pct_positive data found on 20200318 for geography 'state'
  NoDataWarning)
/usr/local/lib/python3.6/dist-packages/covidcast/covidcast.py:391: NoDataWarning: No quidel covid_ag_smoothed_pct_positive data found on 20200319 for geography 'state'
  NoDataWarning)
/usr/local/lib/python3.6/dist-packages/covidcast/covidcast.py:391: NoDataWarning: No quidel covid_ag_smoothed_pct_positive data found on 20200320 for geography 'state'
  NoDataWarning)
/usr/local/lib/python3.6/dist-packages/covidcast/covidcast.py:391: NoDataWarning: No quidel covid_ag_smoothed_pct_positive data found on 20200321 for geography 'state'
  NoDataWarning)
/usr/local/lib/python3.6/dist-packages/covidcast/covidcast.py:391: NoDataWarning: No quidel covid_ag_smoothed_pct_positive data found on 20200322 for geography 'state'
  NoDataWarning)
/usr/local/lib/python3.6/dist-packages/covidcast/covidcast.py:391: NoDataWa

... currently fetching (safegraph, full_time_work_prop) ...
... currently fetching (safegraph, part_time_work_prop) ...
... currently fetching (indicator-combination, confirmed_7dav_incidence_num) ...
... currently fetching (indicator-combination, deaths_7dav_incidence_num) ...
... currently fetching (indicator-combination, confirmed_7dav_incidence_num) ...
... currently fetching (indicator-combination, deaths_7dav_incidence_num) ...
... currently fetching (indicator-combination, confirmed_7dav_incidence_num) ...
... currently fetching (indicator-combination, deaths_7dav_incidence_num) ...


In [ ]:
def calculate_neighbor(state_code, day, typ):
    if len(state_neighbors[state_code]) == 0:
        return 0
    total_val = 0
    total_pop = 0
    index = 6 if typ == 'cases' else 7
    for neighbor in state_neighbors[state_code]:
        state_two_week = two_week_data_list[index].loc[two_week_data_list[index]['state_code'] == neighbor]
        val = 0 if len(state_two_week.loc[state_two_week['date'] == day]) == 0 \
        or state_two_week.loc[state_two_week['date']==day]['value'].tolist()[0] < 0 \
        else state_two_week.loc[state_two_week['date'] == day]['value'].tolist()[0]
        pop = int(pop_df.loc[pop_df['state_code'] == neighbor]['population'].tolist()[0].replace(',', ''))
        
        total_val = total_val + val
        total_pop = total_pop + pop
    
    return total_val/total_pop

In [ ]:
all_data = []

for i in range(0,240):
    
    today = str(date(2020,4,1) + timedelta(days=i))
    two_weeks_ago = str(date(2020,4,1) + timedelta(days=i) - timedelta(days=14))
    three_weeks_ago = str(date(2020,4,1) + timedelta(days=i) - timedelta(days=21))
    four_weeks_ago = str(date(2020,4,1) + timedelta(days=i) - timedelta(days=28))
    
    print('... now working on ' + str(today) + " ...")
    
    #for state in state_dict:
    for state_code in state_dict:

        since_march_31 = i + 1

        lat = latlon_df.loc[latlon_df['code'] == state_code]['lat'].tolist()[0]

        lon = latlon_df.loc[latlon_df['code'] == state_code]['lon'].tolist()[0]
        
        pop = int(pop_df.loc[pop_df['state_code'] == state_code]['population'].tolist()[0].replace(',', ''))
        
        pop_den = float(pop_df.loc[pop_df['state_code'] == state_code]['pop_density'].tolist()[0].replace(',', ''))

        mandate_level = mask_df.loc[mask_df['date'] == two_weeks_ago][state_code].tolist()[0]
        
        state_cases = today_data_list[0].loc[today_data_list[0]['state_code'] == state_code]
        cases = 0 if len(state_cases.loc[state_cases['date'] == today]) == 0 \
        or state_cases.loc[state_cases['date']==today]['value'].tolist()[0] < 0 \
        else int(state_cases.loc[state_cases['date'] == today]['value'].tolist()[0])
        
        state_deaths = today_data_list[1].loc[today_data_list[1]['state_code'] == state_code]
        deaths = 0 if len(state_deaths.loc[state_deaths['date'] == today]) == 0 \
        or state_deaths.loc[state_deaths['date']==today]['value'].tolist()[0] < 0 \
        else int(state_deaths.loc[state_deaths['date'] == today]['value'].tolist()[0])
        
        state_doctor_visits = two_week_data_list[0].loc[two_week_data_list[0]['state_code'] == state_code]
        doctor_visits = 0.0 if len(state_doctor_visits.loc[state_doctor_visits['date'] == two_weeks_ago]) == 0 \
        else state_doctor_visits.loc[state_doctor_visits['date'] == two_weeks_ago]['value'].tolist()[0]
        
        state_google = two_week_data_list[1].loc[two_week_data_list[1]['state_code'] == state_code]
        google = 0.0 if len(state_google.loc[state_google['date'] == two_weeks_ago]) == 0 \
        else state_google.loc[state_google['date'] == two_weeks_ago]['value'].tolist()[0]
        
        state_admissions = two_week_data_list[2].loc[two_week_data_list[2]['state_code'] == state_code]
        admissions = 0.0 if len(state_admissions.loc[state_admissions['date'] == two_weeks_ago]) == 0 \
        else state_admissions.loc[state_admissions['date'] == two_weeks_ago]['value'].tolist()[0]
        
        state_quidel = two_week_data_list[3].loc[two_week_data_list[3]['state_code'] == state_code]
        quidel = 0.0 if len(state_quidel.loc[state_quidel['date'] == two_weeks_ago]) == 0 \
        else state_quidel.loc[state_quidel['date'] == two_weeks_ago]['value'].tolist()[0]
        
        state_full_time = two_week_data_list[4].loc[two_week_data_list[4]['state_code'] == state_code]
        full_time = 0.0 if len(state_full_time.loc[state_full_time['date'] == two_weeks_ago]) == 0 \
        else state_full_time.loc[state_full_time['date'] == two_weeks_ago]['value'].tolist()[0]
        
        state_part_time = two_week_data_list[5].loc[two_week_data_list[5]['state_code'] == state_code]
        part_time = 0.0 if len(state_part_time.loc[state_part_time['date'] == two_weeks_ago]) == 0 \
        else state_part_time.loc[state_part_time['date'] == two_weeks_ago]['value'].tolist()[0]
        
        state_two_week_cases = two_week_data_list[6].loc[two_week_data_list[6]['state_code'] == state_code]
        two_week_cases_avg = 0 if len(state_two_week_cases.loc[state_two_week_cases['date'] == two_weeks_ago]) == 0 \
        or state_two_week_cases.loc[state_two_week_cases['date']==two_weeks_ago]['value'].tolist()[0] < 0 \
        else state_two_week_cases.loc[state_two_week_cases['date'] == two_weeks_ago]['value'].tolist()[0]
        
        state_two_week_deaths = two_week_data_list[7].loc[two_week_data_list[7]['state_code'] == state_code]
        two_week_deaths_avg = 0 if len(state_two_week_deaths.loc[state_two_week_deaths['date'] == two_weeks_ago]) == 0 \
        or state_two_week_deaths.loc[state_two_week_deaths['date']==two_weeks_ago]['value'].tolist()[0] < 0 \
        else state_two_week_deaths.loc[state_two_week_deaths['date'] == two_weeks_ago]['value'].tolist()[0]
        
        state_three_week_cases = three_week_data_list[0].loc[three_week_data_list[0]['state_code'] == state_code]
        three_week_cases_avg = 0 if len(state_three_week_cases.loc[state_three_week_cases['date'] == three_weeks_ago]) == 0 \
        or state_three_week_cases.loc[state_three_week_cases['date']==three_weeks_ago]['value'].tolist()[0] < 0 \
        else state_three_week_cases.loc[state_three_week_cases['date'] == three_weeks_ago]['value'].tolist()[0]
        
        state_three_week_deaths = three_week_data_list[1].loc[three_week_data_list[1]['state_code'] == state_code]
        three_week_deaths_avg = 0 if len(state_three_week_deaths.loc[state_three_week_deaths['date'] == three_weeks_ago]) == 0 \
        or state_three_week_deaths.loc[state_three_week_deaths['date']==three_weeks_ago]['value'].tolist()[0] < 0 \
        else state_three_week_deaths.loc[state_three_week_deaths['date'] == three_weeks_ago]['value'].tolist()[0]
        
        state_four_week_cases = four_week_data_list[0].loc[four_week_data_list[0]['state_code'] == state_code]
        four_week_cases_avg = 0 if len(state_four_week_cases.loc[state_four_week_cases['date'] == four_weeks_ago]) == 0 \
        or state_three_week_cases.loc[state_four_week_cases['date']==four_weeks_ago]['value'].tolist()[0] < 0 \
        else state_four_week_cases.loc[state_four_week_cases['date'] == four_weeks_ago]['value'].tolist()[0]
        
        state_four_week_deaths = four_week_data_list[1].loc[four_week_data_list[1]['state_code'] == state_code]
        four_week_deaths_avg = 0 if len(state_four_week_deaths.loc[state_four_week_deaths['date'] == four_weeks_ago]) == 0 \
        or state_four_week_deaths.loc[state_four_week_deaths['date']==four_weeks_ago]['value'].tolist()[0] < 0 \
        else state_four_week_deaths.loc[state_four_week_deaths['date'] == four_weeks_ago]['value'].tolist()[0]
        
        case_trend = three_week_cases_avg - two_week_cases_avg
        
        death_trend = three_week_deaths_avg - two_week_deaths_avg
        
        neighbor_cases = calculate_neighbor(state_code, two_weeks_ago, 'cases')
        
        neighbor_deaths = calculate_neighbor(state_code, two_weeks_ago, 'deaths')
        
        data = [state_code, state_dict[state_code], pop, pop_den, lat, lon, today, since_march_31, mandate_level, doctor_visits, google,
                admissions, quidel, full_time, part_time, two_week_cases_avg, two_week_deaths_avg,
                three_week_cases_avg, three_week_deaths_avg, four_week_cases_avg, four_week_deaths_avg,
                case_trend, death_trend, neighbor_cases, neighbor_deaths, cases, deaths]
        
        all_data.append(data)

... now working on 2020-04-01 ...
... now working on 2020-04-02 ...
... now working on 2020-04-03 ...
... now working on 2020-04-04 ...
... now working on 2020-04-05 ...
... now working on 2020-04-06 ...
... now working on 2020-04-07 ...
... now working on 2020-04-08 ...
... now working on 2020-04-09 ...
... now working on 2020-04-10 ...
... now working on 2020-04-11 ...
... now working on 2020-04-12 ...
... now working on 2020-04-13 ...
... now working on 2020-04-14 ...
... now working on 2020-04-15 ...
... now working on 2020-04-16 ...
... now working on 2020-04-17 ...
... now working on 2020-04-18 ...
... now working on 2020-04-19 ...
... now working on 2020-04-20 ...
... now working on 2020-04-21 ...
... now working on 2020-04-22 ...
... now working on 2020-04-23 ...
... now working on 2020-04-24 ...
... now working on 2020-04-25 ...
... now working on 2020-04-26 ...
... now working on 2020-04-27 ...
... now working on 2020-04-28 ...
... now working on 2020-04-29 ...
... now workin

In [ ]:
final_df = pd.DataFrame(all_data, columns = ['code','state','pop','pop_den','lat','lon','date','since_march','mandate_level','visits','google',
                                             'admissions','quidel','full','part','2_case_avg','2_death_avg',
                                             '3_case_avg','3_death_avg','4_case_avg','4_death_avg','case_trend',
                                             'death_trend','neighbor_cases','neighbor_deaths','cases','deaths'])
final_df.to_csv('/content/drive/My Drive/MSDA/MKTG881/data/covid19.csv',index=False)